<a href="https://colab.research.google.com/github/tcapelle/mosaic/blob/master/MosaicML_Composer_and_wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--- @wandbcode{mosaicml} -->

<img src="https://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />
<img src="https://raw.githubusercontent.com/mosaicml/composer/dev/docs/source/_static/images/header_dark.svg" width="400" alt="mosaicml" />

<!--- @wandbcode{lit_colab_boris} -->

# Running fast with MosaicML Composer and Weight and Biases

[MosaicML Composer](https://docs.mosaicml.com) is a library for training neural networks better, faster, and cheaper. It contains many state-of-the-art methods for accelerating neural network training and improving generalization, along with an optional Trainer API that makes composing many different enhancements easy.

Coupled with [Weights & Biases integration](https://docs.mosaicml.com/en/v0.5.0/trainer/logging.html), you can quickly train and monitor models for full traceability and reproducibility with only 2 extra lines of code:

```python
from composer.loggers import WandBLogger
wandb_logger = WandBLogger()
```

W&B integration with Composer can automatically:
* log your configuration parameters
* log your losses and metrics
* log gradients and parameter distributions
* log your model
* keep track of your code
* log your system metrics (GPU, CPU, memory, temperature, etc)

### 🛠️ Installation and set-up

We need to install the following libraries:
* [mosaicml-composer](https://docs.mosaicml.com/en/v0.5.0/getting_started/installation.html) to set up and train our models
* [wandb](https://docs.wandb.ai/) to instrument our training

In [39]:
# !pip install wandb mosaicml fastcore

## 🔥 Getting Started with Composer 

Composer gives you access to a set of functions to speedup your models and infuse them with state of the art methods. For instance, you can insert [BlurPool](https://docs.mosaicml.com/en/latest/method_cards/blurpool.html) into your CNN by calling `CF.apply_blurpool(model)` into your PyTorch model. Take a look at all the [functional](https://docs.mosaicml.com/en/latest/functional_api.html) methods available.

In [40]:
import logging
from fastcore.all import *
from composer import functional as CF
import torchvision.models as models

logging.basicConfig(level=logging.INFO)
model = models.resnet50()

# replace some layers with blurpool
CF.apply_blurpool(model);
# replace some layers with squeeze-excite
CF.apply_squeeze_excite(model, latent_channels=64, min_channels=128);

INFO:composer.utils.module_surgery:optimizers was not provided. Be sure to either create the optimizer after
invoking this method, or manually add new parameters to the existing optimizer.
INFO:composer.algorithms.blurpool.blurpool:Applied BlurPool to model ResNet. Model now has 1 BlurMaxPool2d and 6 BlurConv2D layers.
INFO:composer.utils.module_surgery:optimizers was not provided. Be sure to either create the optimizer after
invoking this method, or manually add new parameters to the existing optimizer.


## Using the `Trainer` class with Weights and Biases 🏋️‍♀️

W&B integration with MosaicML-Composer is built into the `Trainer` and can be configured to add extra functionalities through `WandBLogger`:

* logging of Artifacts: Use `log_artifacts=True` to log model checkpoints as `wandb.Artifacts`. You can setup how often by passing an int value to `log_artifacts_every_n_batches` (default = 100)
* you can also pass any parameter that you would pass to `wandb.init` in `init_params` as a dictionary. For example, you could pass `init_params = {"project":"try_mosaicml", "name":"benchmark", "entity":"user_name"}`.

For more details refer to [Logger documentation](https://docs.mosaicml.com/en/latest/api_reference/composer.loggers.wandb_logger.html#composer.loggers.wandb_logger.WandBLogger) and [Wandb docs](https://docs.wandb.ai)

Let's grab fastai's [Imagenette dataset](https://github.com/fastai/imagenette) and decompress it

In [41]:
EPOCHS = 10
BS = 32

In [42]:
if not Path('imagenette2.tgz').exists():
    URL = 'https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz'
    !wget {URL}
    imagenette_path = untar_dir("imagenette2.tgz", Path("."))
else:
    imagenette_path = Path('imagenette2')

In [43]:
import torchvision as tv

class Imagenette:
    def __init__(self, path, train=True, transform=None):
        self.path = Path(path) / ("train" if train else "val")
        self.transform = transform
        self.files = list(path.glob("**/*.JPEG"))
        self.classes = {label.name:i for i, label in enumerate(self.path.iterdir())}
        
    def __getitem__(self, idx):
        file_path = self.files[idx]
        image = self.transform(tv.io.read_image(str(file_path), 
                                             mode=tv.io.image.ImageReadMode.RGB)) 
        label = self.classes[file_path.parent.name]
        return image, label
            
    def __len__(self):
        return len(self.files)

In [44]:
import torch
import torchvision.transforms as T
from torchvision import datasets
from torch.utils.data import DataLoader

from composer import Trainer

In [45]:
imagenet_stats = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
transform = transforms.Compose([T.CenterCrop(256), 
                                T.ConvertImageDtype(torch.float),
                                T.Normalize(*imagenet_stats)
                               ])
train_dataset = Imagenette(imagenette_path, train=True, transform=transform)
eval_dataset = Imagenette(imagenette_path, train=False, transform=transform)

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=BS, num_workers=8)
eval_dataloader = DataLoader(eval_dataset, batch_size=2*BS, num_workers=8)

A simple model and the `DecoupledAdamW` optimizer.

In [46]:
import torchmetrics
import torch.nn.functional as F
from composer.models import ComposerModel
from torchvision.models import resnet18

class ResNet18(ComposerModel):

    def __init__(self, num_classes):
        super().__init__()
        self.model = resnet18(num_classes=num_classes)
        self.train_accuracy = torchmetrics.Accuracy()
        self.val_accuracy = torchmetrics.Accuracy()

    def forward(self, batch): # batch is the output of the dataloader
        # specify how batches are passed through the model
        inputs, _ = batch
        return self.model(inputs)

    def loss(self, outputs, batch):
        # pass batches and `forward` outputs to the loss
        _, targets = batch
        return F.cross_entropy(outputs, targets)
    
    def validate(self, batch):

        inputs, targets = batch
        outputs = self.model(inputs)
        return outputs, targets
    
    def metrics(self, train=False):
        # defines which metrics to use in each phase of training
        return self.train_accuracy if train else self.val_accuracy
    
model = ResNet18(10)

we define the `wandb.init` params here

In [47]:
from composer.loggers import WandBLogger, TQDMLogger

In [48]:
init_params = {"project":"composer", 
               "name":"imagenette_baseline"}

# we pass the to the logger 
wandb_logger = WandBLogger(init_params=init_params)

# we also add progressbar logging
progress_logger = TQDMLogger()

loggers = [progress_logger, wandb_logger]

to tweak what are we logging, we can pass `Callbacks` to the `Trainer` class.

In [49]:
from composer.callbacks import SpeedMonitor, LRMonitor, CheckpointSaver

we include callbacks that measure the model throughput (and the learning rate) and logs them to Weights & Biases. `Callbacks` control what is being logged, whereas loggers specify where the information is being saved. For more information on loggers, see [Logging](https://docs.mosaicml.com/en/latest/trainer/logging.html).

In [50]:
callbacks = [LRMonitor(),    # Logs the learning rate
             SpeedMonitor(), # Logs the training throughput
            ]

we can also create a custom callback to log samples to `Weights and Biases` workspace,

In [51]:
import wandb
from composer import Callback, State, Logger

class LogPredictions(Callback):
    def __init__(self, num_samples=100, seed=1234):
        super().__init__()
        self.num_samples = num_samples
        self.data = []
        
    def eval_batch_end(self, state: State, logger: Logger):
        """Compute predictions per batch and stores them on self.data"""
        
        if state.timer.epoch == state.max_duration: #on last val epoch
            if len(self.data) < self.num_samples:
                n = self.num_samples
                x, y = state.batch_pair
                outputs = state.outputs.argmax(-1)
                data = [[wandb.Image(x_i), y_i, y_pred] for x_i, y_i, y_pred in list(zip(x[:n], y[:n], outputs[:n]))]
                self.data += data
            
    def eval_end(self, state: State, logger: Logger):
        "Create a wandb.Table and logs it"
        columns = ['image', 'ground truth', 'prediction']
        table = wandb.Table(columns=columns, data=self.data[:self.num_samples])
        wandb.log({'sample_table':table}, step=int(state.timer.batch))

In [52]:
callbacks.append(LogPredictions())

then we pass them to the `Trainer`

In [15]:
trainer = Trainer(
    model=model,
    train_dataloader=train_dataloader,
    eval_dataloader=eval_dataloader,
    optimizers=torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-2),
    max_duration=f"{EPOCHS}ep",
    loggers=loggers,
    callbacks=callbacks,
    device="gpu",
    precision="amp",

)
trainer.fit()

INFO:composer.trainer.trainer:Setting seed to 2928117546
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: capecape (use `wandb login --relogin` to force relogin)


Epoch 0:   0%|          | 0/419 [00:00<?, ?it/s]

Epoch 1, Batch 419 (val):   0%|          | 0/210 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/419 [00:00<?, ?it/s]

Epoch 2, Batch 838 (val):   0%|          | 0/210 [00:00<?, ?it/s]

...

Epoch 8:   0%|          | 0/419 [00:00<?, ?it/s]

Epoch 9, Batch 3771 (val):   0%|          | 0/210 [00:00<?, ?it/s]

Epoch 9:   0%|          | 0/419 [00:00<?, ?it/s]

Epoch 10, Batch 4190 (val):   0%|          | 0/210 [00:00<?, ?it/s]

accuracy/val,▁▂▂▁▇▇▆██▆
epoch,▁▂▂▃▄▅▅▆▇▇█
loss/train,██▇▆▅█▄▃▆▅▇▇▅▇▅▄▃▅▄▆▄▄▃▃▅▃▂▂▄▄▄▃▄▂▁▄▁▂▂▃
lr-Adam/group0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
throughput/epoch,▁████▇▇▇██
throughput/step,█▆▅▆▄▄▄▆▄▅▅▅▆▅▄▄▆▅▅▅▅▄▅▄▅▄▅▅▆▁▄▅▅▅▅▅█▇▄▄
trainer/batch_idx,▂▃▆▇▁▄▅▇▂▃▅█▁▄▅▇▂▃▅█▁▃▆▇▂▄▅█▁▃▆▇▂▄▅▇▂▃▆█
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
wall_clock_train,▁▂▃▃▄▅▆▆▇█
accuracy/val,0.52329
epoch,10


## Going 🚀 with the Trainer
We can try some of the magic algorithms from Mosaic

In [60]:
from composer.algorithms import LabelSmoothing, MixUp, ChannelsLast, ColOut, BlurPool
from composer.optim import DecoupledAdamW, CosineAnnealingWithWarmupScheduler

In [61]:
model = ResNet18(num_classes=10)

In [65]:
dadam = DecoupledAdamW(model.parameters(), lr=1e-3)

cosine_annel = CosineAnnealingWithWarmupScheduler('1ep', '1dur')

In [66]:
init_params = {"project":"composer", 
               "name":"imagenette_algos"}

# we pass the to the logger 
wandb_logger = WandBLogger(init_params=init_params)

# we also add progressbar logging
progress_logger = TQDMLogger()

loggers = [progress_logger, wandb_logger]

In [67]:
callbacks = [LRMonitor(),    # Logs the learning rate
             SpeedMonitor(), # Logs the training throughput
             LogPredictions()]

In [68]:
algorithms=[LabelSmoothing()]

In [ ]:
trainer = Trainer(
    model=model,
    train_dataloader=train_dataloader,
    eval_dataloader=eval_dataloader,
    max_duration=f"{EPOCHS}ep",
    loggers=loggers,
    callbacks=callbacks,
    optimizers=dadam,
    schedulers=cosine_annel,
    algorithms=algorithms,
    precision="amp",
    device="gpu",
)
trainer.fit()

INFO:composer.trainer.trainer:Setting seed to 1621805655


Epoch 0:   0%|          | 0/419 [00:00<?, ?it/s]

Epoch 1, Batch 419 (val):   0%|          | 0/210 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/419 [00:00<?, ?it/s]

Epoch 2, Batch 838 (val):   0%|          | 0/210 [00:00<?, ?it/s]

...

Epoch 3:   0%|          | 0/419 [00:00<?, ?it/s]

Epoch 4, Batch 1676 (val):   0%|          | 0/210 [00:00<?, ?it/s]

Epoch 4:   0%|          | 0/419 [00:00<?, ?it/s]

Epoch 5, Batch 2095 (val):   0%|          | 0/210 [00:00<?, ?it/s]

Epoch 5:   0%|          | 0/419 [00:00<?, ?it/s]

# Fastai

In [29]:
import wandb
from fastai.vision.all import *
from fastai.callback.wandb import WandbCallback

In [30]:
dls = ImageDataLoaders.from_folder(imagenette_path, train="train", valid="val", 
                                   bs=BS, val_bs=2*BS, item_tfms=Resize(256))

In [31]:
with wandb.init(project="composer", name="fastai"):
    cbs = [MixedPrecision(), WandbCallback(log_preds=False)]
    learn = cnn_learner(dls, resnet18, metrics=[accuracy], cbs=cbs, pretrained=False)
    learn.fit_one_cycle(EPOCHS, 1e-3)

WandbCallback requires use of "SaveModelCallback" to log best model


epoch,train_loss,valid_loss,accuracy,time
0,2.442298,2.264880,0.340637,00:35
1,1.838694,4.156926,0.269045,00:35
2,1.497490,2.339456,0.339363,00:35
3,1.265657,1.768469,0.474395,00:34
4,0.977236,1.597666,0.553121,00:34
5,0.850229,0.920555,0.712102,00:36
6,0.688579,1.503470,0.570191,00:36
7,0.534455,0.563701,0.817070,00:35
8,0.468598,0.511663,0.837452,00:35
9,0.415594,0.493384,0.844841,00:35


accuracy,▂▁▂▃▄▆▅███
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▂▂▃▄▅▆▇▇██████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▂▁▁▁▁▁
lr_1,▁▂▂▃▄▅▆▇▇██████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▂▁▁▁▁▁
lr_2,▁▂▂▃▄▅▆▇▇██████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▂▁▁▁▁▁
mom_0,██▇▆▅▄▃▂▂▁▁▁▁▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇▇█████
mom_1,██▇▆▅▄▃▂▂▁▁▁▁▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇▇█████
mom_2,██▇▆▅▄▃▂▂▁▁▁▁▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇▇█████
